In [9]:
import torch
#加载数据集
from Dataset import *
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


class Inception(torch.nn.Module):
    #  Inception模块
    # 输入ninput：输入通道数
    # 输入n1：第一个1×1卷积核分支的输出通道数
    # 输入n3pre和n3：第二个3×3卷积核分支的输出通道数
    # 输入n5pre和n5：第三个5×5卷积核分支的输出通道数
    # 输入npool：第四个池化层分支的输出通道数
    def __init__(self, ninput, n1, n3pre, n3, n5pre, n5, npool):
        super(Inception, self).__init__()
        self.b1 = torch.nn.Sequential(
            torch.nn.Conv2d(ninput, n1, kernel_size=1),
            torch.nn.ReLU(inplace=True)
        )
        self.b2 = torch.nn.Sequential(
            torch.nn.Conv2d(ninput, n3pre, kernel_size=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(n3pre, n3, kernel_size=3, padding=1)
        )
        self.b3 = torch.nn.Sequential(
            torch.nn.Conv2d(ninput, n5pre, kernel_size=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(n5pre, n5, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True)
        )
        self.b4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(inplace=True)
        )

    def forward(self, x):
        outpus = [self.b1(x), self.b2(x), self.b3(x), self.b4(x)]
        return torch.cat(outpus, dim=1)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(74, 20, kernel_size=5)

        self.incep1 = Inception(10, 16, 16, 24, 16, 24, 24)
        self.incep2 = Inception(20, 16, 16, 24, 16, 24, 24)

        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1344, 10)

    def forward(self, x):
        in_size = x.size(0)
        out = F.relu(self.mp(self.conv1(x)))
        out = self.incep1(out)
        out = self.conv2(out)
        out = self.mp(out)
        out = F.relu(out)
        out = self.incep2(out)
        out = out.view(in_size, -1)
        out = self.fc(out)
        return out


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.4f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]  # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    print('\nAccuracy: {}/{} ({:.4f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


if __name__ == '__main__':
    BATCH_SIZE = 512  # 大概需要2G的显存
    EPOCHS = 10  # 总共训练批次
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    

    # 加载训练数据
    myTrainData = MyDataSet("ecg_data.csv")
    # 将batch_size设置成50，表示每一次迭代取出50个数据。
    train_loader = DataLoader(dataset=myTrainData, batch_size=50, shuffle=True)

    # 加载测试数据
    myTestData = MyDataSet("ecg_data.csv")
    test_loader = DataLoader(dataset=myTestData, batch_size=50, shuffle=True)

    # for i, img_data in enumerate(train_loader, 1):
    #     images, labels = img_data
    #     print('batch{0}:images shape info-->{1} labels-->{2}'.format(i, images.shape, labels))

    model = Net().to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(1, EPOCHS + 1):
        train(model, DEVICE, train_loader, optimizer, epoch)
        test(model, DEVICE, test_loader)


KeyError: 'symmetry_mean'